In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft2, ifft2
import cv2
from torchvision import datasets, transforms

# Load and preprocess the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
mnist_image, _ = trainset[0]
q_mnist_image = np.abs(mnist_image.numpy()[0])

# Load and process the Lenna image
lenna_image_path = 'C:/Users/ACER/Desktop/lena_256.jpg'
lenna_image = cv2.imread(lenna_image_path, cv2.IMREAD_GRAYSCALE)
if lenna_image is None:
    raise ValueError(f"Error loading Lenna image from path: {lenna_image_path}")
lenna_image = cv2.resize(lenna_image, (28, 28))
q_lenna_image = lenna_image / 255.0

# Define Quantum Fourier Transform (QFT)
def QFT(image):
    return fft2(image)

# Define inverse Quantum Fourier Transform (iQFT)
def iQFT(image):
    return ifft2(image)

# Define padding operation
def pad_to_same_size(image, target_shape):
    pad_height = target_shape[0] - image.shape[0]
    pad_width = target_shape[1] - image.shape[1]
    pad_height_top = pad_height // 2
    pad_height_bottom = pad_height - pad_height_top
    pad_width_left = pad_width // 2
    pad_width_right = pad_width - pad_width_left
    return np.pad(image, ((pad_height_top, pad_height_bottom), (pad_width_left, pad_width_right)), mode='constant')

# Define convolution in frequency domain
def quantum_convolution(image, kernel):
    # Pad image and kernel to the same size
    target_shape = (image.shape[0] + kernel.shape[0] - 1, image.shape[1] + kernel.shape[1] - 1)
    padded_image = pad_to_same_size(image, target_shape)
    padded_kernel = pad_to_same_size(kernel, target_shape)
    
    # Apply QFT
    qft_image = QFT(padded_image)
    qft_kernel = QFT(padded_kernel)
    
    # Element-wise multiplication
    convolved = qft_image * qft_kernel
    
    # Apply inverse QFT
    result = iQFT(convolved)
    
    # Crop the result to the original image size
    result = np.abs(result)
    crop_size = ((target_shape[0] - image.shape[0]) // 2, (target_shape[1] - image.shape[1]) // 2)
    result = result[crop_size[0]:crop_size[0] + image.shape[0], crop_size[1]:crop_size[1] + image.shape[1]]
    
    return result

# Define example kernel
kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])

# Apply quantum convolution on MNIST image
mnist_convolved = quantum_convolution(q_mnist_image, kernel)

# Apply quantum convolution on Lenna image
lenna_convolved = quantum_convolution(q_lenna_image, kernel)

# Visualize the results
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# MNIST
axs[0, 0].imshow(q_mnist_image, cmap='gray')
axs[0, 0].set_title('Original MNIST Image')
axs[0, 1].imshow(mnist_convolved, cmap='gray')
axs[0, 1].set_title('Convolved MNIST Image')

# Lenna
axs[1, 0].imshow(q_lenna_image, cmap='gray')
axs[1, 0].set_title('Original Lenna Image')
axs[1, 1].imshow(lenna_convolved, cmap='gray')
axs[1, 1].set_title('Convolved Lenna Image')

for ax in axs.flat:
    ax.label_outer()

plt.show()


In [ ]:
import torch
from torchvision import datasets, transforms
import numpy as np

# Load MNIST dataset and preprocess
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
mnist_image, _ = trainset[0]
q_mnist_image = np.abs(mnist_image.numpy()[0])
import cv2

lenna_image_path = 'C:/Users/ACER/Desktop/lena_256.jpg'
lenna_image = cv2.imread(lenna_image_path, cv2.IMREAD_GRAYSCALE)
if lenna_image is None:
    raise ValueError(f"Error loading Lenna image from path: {lenna_image_path}")
lenna_image = cv2.resize(lenna_image, (28, 28))
q_lenna_image = lenna_image / 255.0
pepper_image_path = 'C:/Users/ACER/Desktop/download.jpg'
pepper_image = cv2.imread(pepper_image_path, cv2.IMREAD_GRAYSCALE)
if pepper_image is None:
    raise ValueError(f"Error loading Pepper image from path: {pepper_image_path}")
pepper_image = cv2.resize(pepper_image, (28, 28))
q_pepper_image = pepper_image / 255.0
import matplotlib.pyplot as plt

# Define the equations as functions
def tensor_product(N, d, K, Q_in, Q_f):
    return N**6 * d**2 * K**2, 2 * (Q_in + Q_f), 2 * gate_count(Q_in, Q_f)

def composition(N, d, K, Q_in, Q_f_smaller):
    return N**4 * d**2 * K**2, Q_in + Q_f_smaller, gate_count(Q_in, Q_f_smaller) + gate_additional()

def single_kernel_element(N, Q_in):
    return N**4, Q_in, limited_gates()

def complex_kernel_element(N, Q_in):
    return 2 * N**4, Q_in, gate_count_rotations()

def multiple_kernels(M, N, d, K, Q_in, Q_f_kernel):
    return M * N**6 * d**2 * K**2, N * (Q_in + Q_f_kernel), M * gate_count(Q_in, Q_f_kernel)

def stride(N, d, K, S, Q_in, Q_f):
    return (N**6 * d**2 * K**2) // S**2, (Q_in + Q_f // S**2), gate_count(Q_in, Q_f // S**2)

def pooling(N, Q_in, Q_f):
    return N**4, (Q_in + Q_f), gate_pooling()

def bn(N, Q_in, Q_f):
    return N**4, (Q_in + Q_f), gate_normalization()

def weighted_kernels(N, d, K, Q_in, Q_f):
    return N**6 * d**2 * K**2, 2 * (Q_in + Q_f), gate_controlled_rotations()

def grouped_kernels(N, G, d, K, Q_in, Q_f):
    return (N // G)**6 * d**2 * K**2, (N // G) * (Q_in + Q_f // G), (N // G) * gate_count(Q_in, Q_f // G)

def dilated_kernels(N, d, K, Q_in, Q_f):
    return N**6 * d**2 * (K-1)**2, (Q_in + Q_f), gate_count(Q_in, Q_f)

def depthwise_separable_kernels(N, d, K, Q_in, Q_f):
    return N**4 * d + N**2 * d * K**2, (Q_in + d * Q_f), gate_depthwise() + gate_pointwise()

def quc_with_pd_circuit(N, d, K, Q_in, Q_f):
    return N**6 * d**2 * K**2, increased_qubits_pd(), gate_padding() + gate_count(Q_in, Q_f)

def quc_with_pd_using_padded_kernel(N, d, K, Q_in, Q_f):
    return N**6 * d**2 * K**2, (Q_in + Q_f), gate_count(Q_in, Q_f)

def quc_with_pd_and_pre_padded_image(N, d, K, Q_in, padded_Q_f):
    return N**6 * d**2 * K**2, (Q_in + padded_Q_f), gate_count(Q_in, padded_Q_f)

def alternative_quc_with_pd_via_fts(N, d, K, Q_in, Q_f):
    return N**6 * d**2 * K**2, (Q_in + Q_f), gate_ft() + gate_conv_f()

# Placeholder functions for gate counts and other constants
def gate_count(Q_in, Q_f): return Q_in * Q_f
def gate_additional(): return 10
def limited_gates(): return 5
def gate_count_rotations(): return 15
def gate_pooling(): return 20
def gate_normalization(): return 10
def gate_controlled_rotations(): return 25
def gate_depthwise(): return 8
def gate_pointwise(): return 12
def increased_qubits_pd(): return 5
def gate_padding(): return 7
def gate_ft(): return 9
def gate_conv_f(): return 11

# Plotting the results for a specific set of parameters
N = 28
d = 1
K = 3
Q_in = 5
Q_f = 4
Q_f_smaller = 3
Q_f_kernel = 2
S = 2
M = 1
G = 2
padded_Q_f = 6

techniques = [
    ("Tensor product", tensor_product(N, d, K, Q_in, Q_f)),
    ("Composition", composition(N, d, K, Q_in, Q_f_smaller)),
    ("Single kernel element", single_kernel_element(N, Q_in)),
    ("Complex kernel element", complex_kernel_element(N, Q_in)),
    ("Multiple kernels", multiple_kernels(M, N, d, K, Q_in, Q_f_kernel)),
    ("Stride", stride(N, d, K, S, Q_in, Q_f)),
    ("Pooling", pooling(N, Q_in, Q_f)),
    ("BN", bn(N, Q_in, Q_f)),
    ("Weighted kernels", weighted_kernels(N, d, K, Q_in, Q_f)),
    ("Grouped kernels", grouped_kernels(N, G, d, K, Q_in, Q_f)),
    ("Dilated kernels", dilated_kernels(N, d, K, Q_in, Q_f)),
    ("Depthwise separable kernels", depthwise_separable_kernels(N, d, K, Q_in, Q_f)),
    ("QuC with PD circuit", quc_with_pd_circuit(N, d, K, Q_in, Q_f)),
    ("QuC with PD using padded kernel", quc_with_pd_using_padded_kernel(N, d, K, Q_in, Q_f)),
    ("QuC with PD and pre-padded image", quc_with_pd_and_pre_padded_image(N, d, K, Q_in, padded_Q_f)),
    ("Alternative QuC with PD via FTs", alternative_quc_with_pd_via_fts(N, d, K, Q_in, Q_f))
]

time_complexities = [t[1][0] for t in techniques]
qubit_counts = [t[1][1] for t in techniques]
gate_counts = [t[1][2] for t in techniques]
labels = [t[0] for t in techniques]

# Plotting
fig, axs = plt.subplots(3, 1, figsize=(10, 20))
fig.tight_layout(pad=5.0)

axs[0].barh(labels, time_complexities)
axs[0].set_title('Time Complexity (Gate Operations)')
axs[0].set_xlabel('Time Complexity')
axs[0].set_ylabel('Technique')

axs[1].barh(labels, qubit_counts)
axs[1].set_title('Qubit Count')
axs[1].set_xlabel('Qubit Count')
axs[1].set_ylabel('Technique')

axs[2].barh(labels, gate_counts)
axs[2].set_title('Gate Count')
axs[2].set_xlabel('Gate Count')
axs[2].set_ylabel('Technique')

plt.show()


In [ ]:
import torch
from torchvision import datasets, transforms
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load MNIST dataset and preprocess
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
mnist_image, _ = trainset[0]
q_mnist_image = np.abs(mnist_image.numpy()[0])

# Load and process the Lenna image
lenna_image_path = 'C:/Users/ACER/Desktop/lena_256.jpg'
lenna_image = cv2.imread(lenna_image_path, cv2.IMREAD_GRAYSCALE)
if lenna_image is None:
    raise ValueError(f"Error loading Lenna image from path: {lenna_image_path}")
lenna_image = cv2.resize(lenna_image, (28, 28))
q_lenna_image = lenna_image / 255.0

# Load and process the Pepper image
pepper_image_path = 'C:/Users/ACER/Desktop/download.jpg'
pepper_image = cv2.imread(pepper_image_path, cv2.IMREAD_GRAYSCALE)
if pepper_image is None:
    raise ValueError(f"Error loading Pepper image from path: {pepper_image_path}")
pepper_image = cv2.resize(pepper_image, (28, 28))
q_pepper_image = pepper_image / 255.0

# Placeholder functions for gate counts and other constants
def gate_count(Q_in, Q_f): return Q_in * Q_f
def gate_additional(): return 10
def limited_gates(): return 5
def gate_count_rotations(): return 15
def gate_pooling(): return 20
def gate_normalization(): return 10
def gate_controlled_rotations(): return 25
def gate_depthwise(): return 8
def gate_pointwise(): return 12
def increased_qubits_pd(): return 5
def gate_padding(): return 7
def gate_ft(): return 9
def gate_conv_f(): return 11

# Define the equations as functions
def tensor_product(N, d, K, Q_in, Q_f):
    return N**6 * d**2 * K**2, 2 * (Q_in + Q_f), 2 * gate_count(Q_in, Q_f)

def composition(N, d, K, Q_in, Q_f_smaller):
    return N**4 * d**2 * K**2, Q_in + Q_f_smaller, gate_count(Q_in, Q_f_smaller) + gate_additional()

def single_kernel_element(N, Q_in):
    return N**4, Q_in, limited_gates()

def complex_kernel_element(N, Q_in):
    return 2 * N**4, Q_in, gate_count_rotations()

def multiple_kernels(M, N, d, K, Q_in, Q_f_kernel):
    return M * N**6 * d**2 * K**2, N * (Q_in + Q_f_kernel), M * gate_count(Q_in, Q_f_kernel)

def stride(N, d, K, S, Q_in, Q_f):
    return (N**6 * d**2 * K**2) // S**2, (Q_in + Q_f // S**2), gate_count(Q_in, Q_f // S**2)

def pooling(N, Q_in, Q_f):
    return N**4, (Q_in + Q_f), gate_pooling()

def bn(N, Q_in, Q_f):
    return N**4, (Q_in + Q_f), gate_normalization()

def weighted_kernels(N, d, K, Q_in, Q_f):
    return N**6 * d**2 * K**2, 2 * (Q_in + Q_f), gate_controlled_rotations()

def grouped_kernels(N, G, d, K, Q_in, Q_f):
    return (N // G)**6 * d**2 * K**2, (N // G) * (Q_in + Q_f // G), (N // G) * gate_count(Q_in, Q_f // G)

def dilated_kernels(N, d, K, Q_in, Q_f):
    return N**6 * d**2 * (K-1)**2, (Q_in + Q_f), gate_count(Q_in, Q_f)

def depthwise_separable_kernels(N, d, K, Q_in, Q_f):
    return N**4 * d + N**2 * d * K**2, (Q_in + d * Q_f), gate_depthwise() + gate_pointwise()

def quc_with_pd_circuit(N, d, K, Q_in, Q_f):
    return N**6 * d**2 * K**2, increased_qubits_pd(), gate_padding() + gate_count(Q_in, Q_f)

def quc_with_pd_using_padded_kernel(N, d, K, Q_in, Q_f):
    return N**6 * d**2 * K**2, (Q_in + Q_f), gate_count(Q_in, Q_f)

def quc_with_pd_and_pre_padded_image(N, d, K, Q_in, padded_Q_f):
    return N**6 * d**2 * K**2, (Q_in + padded_Q_f), gate_count(Q_in, padded_Q_f)

def alternative_quc_with_pd_via_fts(N, d, K, Q_in, Q_f):
    return N**6 * d**2 * K**2, (Q_in + Q_f), gate_ft() + gate_conv_f()

# Plotting the results for a specific set of parameters
N = 28
d = 1
K = 3
Q_in = 5
Q_f = 4
Q_f_smaller = 3
Q_f_kernel = 2
S = 2
M = 1
G = 2
padded_Q_f = 6

# Compute metrics for each image
def compute_metrics(image_name):
    techniques = [
        ("Tensor product", tensor_product(N, d, K, Q_in, Q_f)),
        ("Composition", composition(N, d, K, Q_in, Q_f_smaller)),
        ("Single kernel element", single_kernel_element(N, Q_in)),
        ("Complex kernel element", complex_kernel_element(N, Q_in)),
        ("Multiple kernels", multiple_kernels(M, N, d, K, Q_in, Q_f_kernel)),
        ("Stride", stride(N, d, K, S, Q_in, Q_f)),
        ("Pooling", pooling(N, Q_in, Q_f)),
        ("BN", bn(N, Q_in, Q_f)),
        ("Weighted kernels", weighted_kernels(N, d, K, Q_in, Q_f)),
        ("Grouped kernels", grouped_kernels(N, G, d, K, Q_in, Q_f)),
        ("Dilated kernels", dilated_kernels(N, d, K, Q_in, Q_f)),
        ("Depthwise separable kernels", depthwise_separable_kernels(N, d, K, Q_in, Q_f)),
        ("QuC with PD circuit", quc_with_pd_circuit(N, d, K, Q_in, Q_f)),
        ("QuC with PD using padded kernel", quc_with_pd_using_padded_kernel(N, d, K, Q_in, Q_f)),
        ("QuC with PD and pre-padded image", quc_with_pd_and_pre_padded_image(N, d, K, Q_in, padded_Q_f)),
        ("Alternative QuC with PD via FTs", alternative_quc_with_pd_via_fts(N, d, K, Q_in, Q_f))
    ]
    
    time_complexities = [t[1][0] for t in techniques]
    qubit_counts = [t[1][1] for t in techniques]
    gate_counts = [t[1][2] for t in techniques]
    labels = [f"{image_name} - {t[0]}" for t in techniques]
    
    return labels, time_complexities, qubit_counts, gate_counts

mnist_labels, mnist_time_complexities, mnist_qubit_counts, mnist_gate_counts = compute_metrics('MNIST')
lenna_labels, lenna_time_complexities, lenna_qubit_counts, lenna_gate_counts = compute_metrics('Lenna')
pepper_labels, pepper_time_complexities, pepper_qubit_counts, pepper_gate_counts = compute_metrics('Pepper')

# Combine all labels and metrics
all_labels = mnist_labels + lenna_labels + pepper_labels
all_time_complexities = mnist_time_complexities + lenna_time_complexities + pepper_time_complexities
all_qubit_counts = mnist_qubit_counts + lenna_qubit_counts + pepper_qubit_counts
all_gate_counts = mnist_gate_counts + lenna_gate_counts + pepper_gate_counts

# Plotting
fig, axs = plt.subplots(3, 1, figsize=(15, 30))
fig.tight_layout(pad=5.0)

axs[0].barh(all_labels, all_time_complexities)
axs[0].set_title('Time Complexity (Gate Operations)')
axs[0].set_xlabel('Time Complexity')
axs[0].set_ylabel('Technique')

axs[1].barh(all_labels, all_qubit_counts)
axs[1].set_title('Qubit Count')
axs[1].set_xlabel('Qubit Count')
axs[1].set_ylabel('Technique')

axs[2].barh(all_labels, all_gate_counts)
axs[2].set_title('Gate Count')
axs[2].set_xlabel('Gate Count')
axs[2].set_ylabel('Technique')


plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
import pandas as pd

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64*7*7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 64*7*7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load MNIST dataset and preprocess
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

def train_and_evaluate(model, trainloader, testloader, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Training
    start_time = time.time()
    model.train()
    for epoch in range(1):  # Use more epochs for actual training
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    training_time = time.time() - start_time
    
    # Evaluation
    model.eval()
    y_true, y_pred = [], []
    start_time = time.time()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    inference_time = time.time() - start_time

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    return accuracy, precision, recall, f1, training_time, inference_time

# Initialize model
model = SimpleCNN()

# Run on MNIST
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mnist_metrics = train_and_evaluate(model, trainloader, testloader, device)

# For simplicity, assuming similar metrics are calculated for Lenna and Pepper
# You will need to adjust the preprocessing, model, and training methods for these images

# Dummy data for Lenna and Pepper (replace with actual computations)
lenna_metrics = (0.85, 0.84, 0.85, 0.84, 50, 0.1)  # Example values
pepper_metrics = (0.80, 0.78, 0.79, 0.78, 55, 0.12)  # Example values

# Create a DataFrame to display the results
metrics_df = pd.DataFrame({
    'Method': ['MNIST', 'Lenna', 'Pepper'],
    'Accuracy': [mnist_metrics[0], lenna_metrics[0], pepper_metrics[0]],
    'Precision': [mnist_metrics[1], lenna_metrics[1], pepper_metrics[1]],
    'Recall': [mnist_metrics[2], lenna_metrics[2], pepper_metrics[2]],
    'F1 Score': [mnist_metrics[3], lenna_metrics[3], pepper_metrics[3]],
    'Training Time (s)': [mnist_metrics[4], lenna_metrics[4], pepper_metrics[4]],
    'Inference Time (s)': [mnist_metrics[5], lenna_metrics[5], pepper_metrics[5]]
})

print(metrics_df)

# Save to CSV file
metrics_df.to_csv('image_datasets_metrics.csv', index=False)



In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from torchvision import datasets, transforms

# Function for quantum convolution with complex kernel element
def quantum_convolution(q_pixel, q_kernel_element, alpha=1):
    return alpha * (q_pixel * q_kernel_element)

# Initial convolution for quantum pipeline
def initial_quantum_convolution(q_pixel, q_kernel_element):
    return np.kron(q_pixel, q_kernel_element)

# Hermitian conjugate operation
def hermitian_conjugate(q_matrix):
    return np.conjugate(q_matrix.T)

# Quantum convolution with complex coefficient alpha
def quantum_convolution_complex(q_pixel, q_kernel_element, alpha):
    return alpha * quantum_convolution(q_pixel, q_kernel_element)

# Function to apply Quantum Fourier Transform (QFT)
def apply_qft(q_matrix):
    N = q_matrix.shape[0]
    qft_matrix = np.fft.fftshift(np.fft.fft2(q_matrix))
    return (1 / np.sqrt(N)) * np.exp(-2j * np.pi * np.arange(N)[:, None] * np.arange(N) / N) @ qft_matrix

# Function for image compression
def image_compression(q_matrix):
    U = np.eye(q_matrix.shape[0])  # Identity matrix for simplification
    return U @ apply_qft(q_matrix) @ hermitian_conjugate(U)

# Function to visualize quantum convolution results and their frequency graphs
def visualize_quantum_convolution(q_image, q_kernel, alpha):
    # Perform initial convolution
    initial_conv = initial_quantum_convolution(q_image, q_kernel)

    # Apply QFT
    qft_result = apply_qft(initial_conv)

    # Perform image compression
    compressed_image = image_compression(initial_conv)

    # Hermitian conjugate operation
    hermitian_result = hermitian_conjugate(compressed_image)

    # Plot the results
    fig, axs = plt.subplots(2, 4, figsize=(20, 10))

    # Original quantum image
    axs[0, 0].imshow(np.abs(q_image), cmap='plasma', norm=LogNorm())
    axs[0, 0].set_title('Original Quantum Image')

    # Initial convolution
    axs[0, 1].imshow(np.abs(initial_conv), cmap='plasma', norm=LogNorm())
    axs[0, 1].set_title('Initial Convolution')

    # QFT result
    axs[0, 2].imshow(np.abs(qft_result), cmap='plasma', norm=LogNorm())
    axs[0, 2].set_title('QFT Result')

    # Compressed image
    axs[0, 3].imshow(np.abs(compressed_image), cmap='plasma', norm=LogNorm())
    axs[0, 3].set_title('Compressed Image')

    # Frequency graphs
    axs[1, 0].plot(np.abs(np.fft.fftshift(np.fft.fft2(q_image)).flatten()))
    axs[1, 0].set_title('Frequency of Original Image')

    axs[1, 1].plot(np.abs(np.fft.fftshift(np.fft.fft2(initial_conv)).flatten()))
    axs[1, 1].set_title('Frequency of Initial Convolution')

    axs[1, 2].plot(np.abs(np.fft.fftshift(np.fft.fft2(qft_result)).flatten()))
    axs[1, 2].set_title('Frequency of QFT Result')

    axs[1, 3].plot(np.abs(np.fft.fftshift(np.fft.fft2(compressed_image)).flatten()))
    axs[1, 3].set_title('Frequency of Compressed Image')

    plt.tight_layout()
    plt.show()

# Load the MNIST dataset using PyTorch
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
image, _ = trainset[0]
q_image = np.abs(image.numpy()[0])

# Parameters for quantum convolution
q_kernel_size = 3
alpha = 1 + 1j  # Complex coefficient alpha

# Generate random quantum kernel element
q_kernel = np.random.rand(q_kernel_size, q_kernel_size) + 1j * np.random.rand(q_kernel_size, q_kernel_size)

# Visualize quantum convolution results
visualize_quantum_convolution(q_image, q_kernel, alpha)


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms

# Define functions for quantum operations
def initial_convolution(q_p, q_ki):
    c_ij = np.random.rand(q_ki.shape[0], q_ki.shape[1])  # Random complex coefficients for kernel elements
    d_kl = np.random.rand(q_p.shape[0], q_p.shape[1])    # Random complex coefficients for quantum image pixels
    Qc = np.kron(c_ij, q_p) @ np.kron(d_kl, q_p)
    return Qc

def quantum_image_compression(q_i, U):
    U_dagger = np.conjugate(U.T)
    q_compressed = np.kron(U, q_i) @ np.kron(U_dagger, q_i)
    return q_compressed

def quantum_image_denoising(q_noisy_image, q_n, F_inv, F):
    q_denoised = F_inv @ (F @ (q_noisy_image - q_n))
    return q_denoised

def hermitian_conjugate(Qc, q_ki, q_p):
    q_ki_dagger = np.conjugate(q_ki.T)
    Qc_dagger = np.kron(q_ki_dagger, q_p).T
    return Qc_dagger

def tensor_associative_property(q_ki, q_p):
    q_ki_dagger = np.conjugate(q_ki.T)
    result = np.kron(q_ki, q_p) @ np.kron(q_ki_dagger, np.eye(q_p.shape[0]))
    return result

# Perform Fourier Transform
def fourier_transform(image):
    return np.fft.fftshift(np.fft.fft2(image))

# Load the MNIST dataset using PyTorch
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
image, _ = trainset[0]
q_image = np.abs(image.numpy()[0])

# Define a sample quantum kernel
q_ki = np.random.rand(28, 28)

# Initial convolution
initial_conv = initial_convolution(q_image, q_ki)

# Quantum image compression
U = np.random.rand(28, 28)
compressed_image = quantum_image_compression(q_image, U)

# Quantum image denoising
F = np.random.rand(28, 28)
F_inv = np.linalg.inv(F)
q_noisy_image = q_image + 0.1 * np.random.rand(28, 28)  # Adding some noise
q_n = 0.1 * np.random.rand(28, 28)  # Noise
denoised_image = quantum_image_denoising(q_noisy_image, q_n, F_inv, F)

# Hermitian conjugate
Qc_dagger = hermitian_conjugate(initial_conv, q_ki, q_image)

# Associative property of the tensor product
associative_result = tensor_associative_property(q_ki, q_image)

# Perform Fourier Transform for frequency graphs
original_freq = np.log(np.abs(fourier_transform(q_image)) + 1)
initial_conv_freq = np.log(np.abs(fourier_transform(initial_conv)) + 1)
compressed_freq = np.log(np.abs(fourier_transform(compressed_image)) + 1)
noisy_freq = np.log(np.abs(fourier_transform(q_noisy_image)) + 1)
denoised_freq = np.log(np.abs(fourier_transform(denoised_image)) + 1)
hermitian_conj_freq = np.log(np.abs(fourier_transform(Qc_dagger)) + 1)

# Plotting the results
fig, axs = plt.subplots(3, 4, figsize=(20, 15))

# Original Quantum Image
axs[0, 0].imshow(q_image, cmap='viridis')
axs[0, 0].set_title('Original Quantum Image')
axs[0, 1].imshow(original_freq, cmap='viridis')
axs[0, 1].set_title('Frequency of Original Image')

# Initial Convolution
axs[0, 2].imshow(np.abs(initial_conv), cmap='viridis')
axs[0, 2].set_title('Initial Convolution')
axs[0, 3].imshow(initial_conv_freq, cmap='viridis')
axs[0, 3].set_title('Frequency of Initial Convolution')

# Compressed Image
axs[1, 0].imshow(np.abs(compressed_image), cmap='viridis')
axs[1, 0].set_title('Compressed Image')
axs[1, 1].imshow(compressed_freq, cmap='viridis')
axs[1, 1].set_title('Frequency of Compressed Image')

# Noisy Quantum Image
axs[1, 2].imshow(np.abs(q_noisy_image), cmap='viridis')
axs[1, 2].set_title('Noisy Quantum Image')
axs[1, 3].imshow(noisy_freq, cmap='viridis')
axs[1, 3].set_title('Frequency of Noisy Image')

# Denoised Quantum Image
axs[2, 0].imshow(np.abs(denoised_image), cmap='viridis')
axs[2, 0].set_title('Denoised Quantum Image')
axs[2, 1].imshow(denoised_freq, cmap='viridis')
axs[2, 1].set_title('Frequency of Denoised Image')

# Hermitian Conjugate Result
axs[2, 2].imshow(np.abs(Qc_dagger), cmap='viridis')
axs[2, 2].set_title('Hermitian Conjugate')
axs[2, 3].imshow(hermitian_conj_freq, cmap='viridis')
axs[2, 3].set_title('Frequency of Hermitian Conjugate')

plt.tight_layout()
plt.show()


In [ ]:
# Install OpenCV
!pip install opencv-python

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from PIL import Image
import cv2

# Define functions for quantum operations
def initial_convolution(q_p, q_ki):
    c_ij = np.random.rand(q_ki.shape[0], q_ki.shape[1])  # Random complex coefficients for kernel elements
    d_kl = np.random.rand(q_p.shape[0], q_p.shape[1])    # Random complex coefficients for quantum image pixels
    Qc = np.kron(c_ij, q_p) @ np.kron(d_kl, q_p)
    return Qc

def quantum_image_compression(q_i, U):
    U_dagger = np.conjugate(U.T)
    q_compressed = np.kron(U, q_i) @ np.kron(U_dagger, q_i)
    return q_compressed

def quantum_image_denoising(q_noisy_image, q_n, F_inv, F):
    q_denoised = F_inv @ (F @ (q_noisy_image - q_n))
    return q_denoised

def hermitian_conjugate(Qc, q_ki, q_p):
    q_ki_dagger = np.conjugate(q_ki.T)
    Qc_dagger = np.kron(q_ki_dagger, q_p).T
    return Qc_dagger

def tensor_associative_property(q_ki, q_p):
    q_ki_dagger = np.conjugate(q_ki.T)
    result = np.kron(q_ki, q_p) @ np.kron(q_ki_dagger, np.eye(q_p.shape[0]))
    return result

# Perform Fourier Transform
def fourier_transform(image):
    return np.fft.fftshift(np.fft.fft2(image))

# Load the MNIST dataset using PyTorch
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
image, _ = trainset[0]
q_image = np.abs(image.numpy()[0])

# Load the Lenna image
lenna_image = cv2.imread('C:/Users/ACER/Desktop/lena_256.jpg', cv2.IMREAD_GRAYSCALE)
lenna_image = cv2.resize(lenna_image, (28, 28))
q_lenna_image = lenna_image / 255.0  # Normalize the image

# Define a sample quantum kernel
q_ki = np.random.rand(28, 28)

# Initial convolution for MNIST image
initial_conv_mnist = initial_convolution(q_image, q_ki)

# Initial convolution for Lenna image
initial_conv_lenna = initial_convolution(q_lenna_image, q_ki)

# Quantum image compression for MNIST image
U = np.random.rand(28, 28)
compressed_image_mnist = quantum_image_compression(q_image, U)

# Quantum image compression for Lenna image
compressed_image_lenna = quantum_image_compression(q_lenna_image, U)

# Quantum image denoising for MNIST image
F = np.random.rand(28, 28)
F_inv = np.linalg.inv(F)
q_noisy_image_mnist = q_image + 0.1 * np.random.rand(28, 28)  # Adding some noise
q_n = 0.1 * np.random.rand(28, 28)  # Noise
denoised_image_mnist = quantum_image_denoising(q_noisy_image_mnist, q_n, F_inv, F)

# Quantum image denoising for Lenna image
q_noisy_image_lenna = q_lenna_image + 0.1 * np.random.rand(28, 28)  # Adding some noise
denoised_image_lenna = quantum_image_denoising(q_noisy_image_lenna, q_n, F_inv, F)

# Hermitian conjugate for MNIST image
Qc_dagger_mnist = hermitian_conjugate(initial_conv_mnist, q_ki, q_image)

# Hermitian conjugate for Lenna image
Qc_dagger_lenna = hermitian_conjugate(initial_conv_lenna, q_ki, q_lenna_image)

# Associative property of the tensor product for MNIST image
associative_result_mnist = tensor_associative_property(q_ki, q_image)

# Associative property of the tensor product for Lenna image
associative_result_lenna = tensor_associative_property(q_ki, q_lenna_image)

# Perform Fourier Transform for frequency graphs
original_freq_mnist = np.log(np.abs(fourier_transform(q_image)) + 1)
initial_conv_freq_mnist = np.log(np.abs(fourier_transform(initial_conv_mnist)) + 1)
compressed_freq_mnist = np.log(np.abs(fourier_transform(compressed_image_mnist)) + 1)
noisy_freq_mnist = np.log(np.abs(fourier_transform(q_noisy_image_mnist)) + 1)
denoised_freq_mnist = np.log(np.abs(fourier_transform(denoised_image_mnist)) + 1)
hermitian_conj_freq_mnist = np.log(np.abs(fourier_transform(Qc_dagger_mnist)) + 1)

original_freq_lenna = np.log(np.abs(fourier_transform(q_lenna_image)) + 1)
initial_conv_freq_lenna = np.log(np.abs(fourier_transform(initial_conv_lenna)) + 1)
compressed_freq_lenna = np.log(np.abs(fourier_transform(compressed_image_lenna)) + 1)
noisy_freq_lenna = np.log(np.abs(fourier_transform(q_noisy_image_lenna)) + 1)
denoised_freq_lenna = np.log(np.abs(fourier_transform(denoised_image_lenna)) + 1)
hermitian_conj_freq_lenna = np.log(np.abs(fourier_transform(Qc_dagger_lenna)) + 1)

# Plotting the results
fig, axs = plt.subplots(6, 4, figsize=(20, 30))

# Original MNIST Quantum Image
axs[0, 0].imshow(q_image, cmap='viridis')
axs[0, 0].set_title('Original MNIST Quantum Image')
axs[0, 1].imshow(original_freq_mnist, cmap='viridis')
axs[0, 1].set_title('Frequency of Original MNIST Image')

# Initial Convolution for MNIST image
axs[0, 2].imshow(np.abs(initial_conv_mnist), cmap='viridis')
axs[0, 2].set_title('Initial Convolution (MNIST)')
axs[0, 3].imshow(initial_conv_freq_mnist, cmap='viridis')
axs[0, 3].set_title('Frequency of Initial Convolution (MNIST)')

# Compressed MNIST Image
axs[1, 0].imshow(np.abs(compressed_image_mnist), cmap='viridis')
axs[1, 0].set_title('Compressed MNIST Image')
axs[1, 1].imshow(compressed_freq_mnist, cmap='viridis')
axs[1, 1].set_title('Frequency of Compressed MNIST Image')

# Noisy MNIST Quantum Image
axs[1, 2].imshow(np.abs(q_noisy_image_mnist), cmap='viridis')
axs[1, 2].set_title('Noisy MNIST Quantum Image')
axs[1, 3].imshow(noisy_freq_mnist, cmap='viridis')
axs[1, 3].set_title('Frequency of Noisy MNIST Image')

# Denoised MNIST Quantum Image
axs[2, 0].imshow(np.abs(denoised_image_mnist), cmap='viridis')
axs[2, 0].set_title('Denoised MNIST Quantum Image')
axs[2, 1].imshow(denoised_freq_mnist, cmap='viridis')
axs[2, 1].set_title('Frequency of Denoised MNIST Image')

# Hermitian Conjugate for MNIST image
axs[2, 2].imshow(np.abs(Qc_dagger_mnist), cmap='viridis')
axs[2, 2].set_title('Hermitian Conjugate (MNIST)')
axs[2, 3].imshow(hermitian_conj_freq_mnist, cmap='viridis')
axs[2, 3].set_title('Frequency of Hermitian Conjugate (MNIST)')

# Original Lenna Quantum Image
axs[3, 0].imshow(q_lenna_image, cmap='viridis')
axs[3, 0].set_title('Original Lenna Quantum Image')
axs[3, 1].imshow(original_freq_lenna, cmap='viridis')
axs[3, 1].set_title('Frequency of Original Lenna Image')

# Initial Convolution for Lenna image
axs[3, 2].imshow(np.abs(initial_conv_lenna), cmap='viridis')
axs[3, 2].set_title('Initial Convolution (Lenna)')
axs[3, 3].imshow(initial_conv_freq_lenna, cmap='viridis')
axs[3, 3].set_title('Frequency of Initial Convolution (Lenna)')

# Compressed Lenna Image
axs[4, 0].imshow(np.abs(compressed_image_lenna), cmap='viridis')
axs[4, 0].set_title('Compressed Lenna Image')
axs[4, 1].imshow(compressed_freq_lenna, cmap='viridis')
axs[4, 1].set_title('Frequency of Compressed Lenna Image')

# Noisy Lenna Quantum Image
axs[4, 2].imshow(np.abs(q_noisy_image_lenna), cmap='viridis')
axs[4, 2].set_title('Noisy Lenna Quantum Image')
axs[4, 3].imshow(noisy_freq_lenna, cmap='viridis')
axs[4, 3].set_title('Frequency of Noisy Lenna Image')

# Denoised Lenna Quantum Image
axs[5, 0].imshow(np.abs(denoised_image_lenna), cmap='viridis')
axs[5, 0].set_title('Denoised Lenna Quantum Image')
axs[5, 1].imshow(denoised_freq_lenna, cmap='viridis')
axs[5, 1].set_title('Frequency of Denoised Lenna Image')

# Hermitian Conjugate for Lenna image
axs[5, 2].imshow(np.abs(Qc_dagger_lenna), cmap='viridis')
axs[5, 2].set_title('Hermitian Conjugate (Lenna)')
axs[5, 3].imshow(hermitian_conj_freq_lenna, cmap='viridis')
axs[5, 3].set_title('Frequency of Hermitian Conjugate (Lenna)')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from PIL import Image
import cv2

def quantum_convolution(q_i, q_kernels, alpha, beta):
    n = len(q_kernels)
    Qc_sum = np.zeros_like(q_i, dtype=np.complex128)
    for m in range(n):
        kernel = q_kernels[m]
        alpha_m = alpha[m]
        beta_m = beta[m]
        kernel_size = kernel.shape
        padded_image = np.pad(q_i, [(kernel_size[0]//2, kernel_size[0]//2), (kernel_size[1]//2, kernel_size[1]//2)], mode='constant')
        conv_result = np.zeros_like(q_i, dtype=np.complex128)
        for i in range(q_i.shape[0]):
            for j in range(q_i.shape[1]):
                conv_result[i, j] = np.sum(
                    padded_image[i:i+kernel_size[0], j:j+kernel_size[1]] * kernel
                )
        Qc_sum += alpha_m * beta_m * conv_result
    return Qc_sum

def quantum_convolution_stride(q_i, q_k, stride):
    q_i_stride = q_i[::stride, ::stride]
    kernel_size = q_k.shape[0]
    padded_image = np.pad(q_i_stride, [(kernel_size//2, kernel_size//2), (kernel_size//2, kernel_size//2)], mode='constant')
    Qc_stride = np.zeros_like(q_i_stride, dtype=np.complex128)
    for i in range(q_i_stride.shape[0]):
        for j in range(q_i_stride.shape[1]):
            Qc_stride[i, j] = np.sum(
                padded_image[i:i+kernel_size, j:j+kernel_size] * q_k
            )
    return Qc_stride

def pad_image(q_i, pad_width):
    return np.pad(q_i, pad_width, mode='constant')

def quantum_convolution_pad(q_i, q_k, pad_width):
    q_i_padded = pad_image(q_i, ((pad_width, pad_width), (pad_width, pad_width)))
    kernel_size = q_k.shape[0]
    Qc_pad = np.zeros_like(q_i_padded, dtype=np.complex128)
    for i in range(q_i.shape[0]):
        for j in range(q_i.shape[1]):
            Qc_pad[i, j] = np.sum(
                q_i_padded[i:i+kernel_size, j:j+kernel_size] * q_k
            )
    return Qc_pad

def batch_norm(q_i, epsilon=1e-5):
    mu = np.mean(q_i)
    sigma2 = np.var(q_i)
    q_bn = (q_i - mu) / np.sqrt(sigma2 + epsilon)
    return q_bn

def quantum_convolution_bn(q_i, q_k, stride, pool_size):
    q_i_bn = batch_norm(q_i)
    q_i_pooled = q_i_bn[:pool_size, :pool_size]
    Qc_bn_pool = quantum_convolution_stride(q_i_pooled, q_k, stride)
    return Qc_bn_pool

def fourier_transform(image):
    return np.fft.fftshift(np.fft.fft2(image))

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
image, _ = trainset[0]
q_image = np.abs(image.numpy()[0])

lenna_image = cv2.imread('C:/Users/ACER/Desktop/lena_256.jpg', cv2.IMREAD_GRAYSCALE)
lenna_image = cv2.resize(lenna_image, (28, 28))
q_lenna_image = lenna_image / 255.0

q_kernels = [np.random.rand(3, 3) for _ in range(3)]
alpha = [np.random.rand() + 1j * np.random.rand() for _ in range(3)]
beta = [np.random.rand() + 1j * np.random.rand() for _ in range(3)]

qc_mnist = quantum_convolution(q_image, q_kernels, alpha, beta)
qc_stride_mnist = quantum_convolution_stride(q_image, q_kernels[0], stride=2)
qc_pad_mnist = quantum_convolution_pad(q_image, q_kernels[0], pad_width=2)
qc_bn_pool_mnist = quantum_convolution_bn(q_image, q_kernels[0], stride=2, pool_size=14)

qc_lenna = quantum_convolution(q_lenna_image, q_kernels, alpha, beta)
qc_stride_lenna = quantum_convolution_stride(q_lenna_image, q_kernels[0], stride=2)
qc_pad_lenna = quantum_convolution_pad(q_lenna_image, q_kernels[0], pad_width=2)
qc_bn_pool_lenna = quantum_convolution_bn(q_lenna_image, q_kernels[0], stride=2, pool_size=14)

fig, axs = plt.subplots(4, 2, figsize=(20, 20))

axs[0, 0].imshow(q_image, cmap='viridis')
axs[0, 0].set_title('Original MNIST Quantum Image')
axs[0, 1].imshow(q_lenna_image, cmap='viridis')
axs[0, 1].set_title('Original Lenna Image')

axs[1, 0].imshow(np.abs(qc_mnist), cmap='viridis')
axs[1, 0].set_title('Quantum Convolution with Multiple Kernels (MNIST)')
axs[1, 1].imshow(np.abs(qc_lenna), cmap='viridis')
axs[1, 1].set_title('Quantum Convolution with Multiple Kernels (Lenna)')

axs[2, 0].imshow(np.abs(qc_stride_mnist), cmap='viridis')
axs[2, 0].set_title('Quantum Convolution with Stride (MNIST)')
axs[2, 1].imshow(np.abs(qc_stride_lenna), cmap='viridis')
axs[2, 1].set_title('Quantum Convolution with Stride (Lenna)')

axs[3, 0].imshow(np.abs(qc_pad_mnist), cmap='viridis')
axs[3, 0].set_title('Quantum Convolution with Padding (MNIST)')
axs[3, 1].imshow(np.abs(qc_pad_lenna), cmap='viridis')
axs[3, 1].set_title('Quantum Convolution with Padding (Lenna)')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from PIL import Image
import cv2

def quantum_convolution(q_i, q_kernels, alpha, beta):
    n = len(q_kernels)
    Qc_sum = np.zeros_like(q_i, dtype=np.complex128)
    for m in range(n):
        kernel = q_kernels[m]
        alpha_m = alpha[m]
        beta_m = beta[m]
        kernel_size = kernel.shape
        padded_image = np.pad(q_i, [(kernel_size[0]//2, kernel_size[0]//2), (kernel_size[1]//2, kernel_size[1]//2)], mode='constant')
        conv_result = np.zeros_like(q_i, dtype=np.complex128)
        for i in range(q_i.shape[0]):
            for j in range(q_i.shape[1]):
                conv_result[i, j] = np.sum(
                    padded_image[i:i+kernel_size[0], j:j+kernel_size[1]] * kernel
                )
        Qc_sum += alpha_m * beta_m * conv_result
    return Qc_sum

def quantum_convolution_stride(q_i, q_k, stride):
    q_i_stride = q_i[::stride, ::stride]
    kernel_size = q_k.shape[0]
    padded_image = np.pad(q_i_stride, [(kernel_size//2, kernel_size//2), (kernel_size//2, kernel_size//2)], mode='constant')
    Qc_stride = np.zeros_like(q_i_stride, dtype=np.complex128)
    for i in range(q_i_stride.shape[0]):
        for j in range(q_i_stride.shape[1]):
            Qc_stride[i, j] = np.sum(
                padded_image[i:i+kernel_size, j:j+kernel_size] * q_k
            )
    return Qc_stride

def pad_image(q_i, pad_width):
    return np.pad(q_i, pad_width, mode='constant')

def quantum_convolution_pad(q_i, q_k, pad_width):
    q_i_padded = pad_image(q_i, ((pad_width, pad_width), (pad_width, pad_width)))
    kernel_size = q_k.shape[0]
    Qc_pad = np.zeros_like(q_i_padded, dtype=np.complex128)
    for i in range(q_i.shape[0]):
        for j in range(q_i.shape[1]):
            Qc_pad[i, j] = np.sum(
                q_i_padded[i:i+kernel_size, j:j+kernel_size] * q_k
            )
    return Qc_pad

def batch_norm(q_i, epsilon=1e-5):
    mu = np.mean(q_i)
    sigma2 = np.var(q_i)
    q_bn = (q_i - mu) / np.sqrt(sigma2 + epsilon)
    return q_bn

def quantum_convolution_bn(q_i, q_k, stride, pool_size):
    q_i_bn = batch_norm(q_i)
    q_i_pooled = q_i_bn[:pool_size, :pool_size]
    Qc_bn_pool = quantum_convolution_stride(q_i_pooled, q_k, stride)
    return Qc_bn_pool

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
image, _ = trainset[0]
q_image = np.abs(image.numpy()[0])

lenna_image = cv2.imread('C:/Users/ACER/Desktop/lena_256.jpg', cv2.IMREAD_GRAYSCALE)
lenna_image = cv2.resize(lenna_image, (28, 28))
q_lenna_image = lenna_image / 255.0

download_image = cv2.imread('C:/Users/ACER/Desktop/download.jpg', cv2.IMREAD_GRAYSCALE)
download_image = cv2.resize(download_image, (28, 28))
q_download_image = download_image / 255.0

q_kernels = [np.random.rand(3, 3) for _ in range(3)]
alpha = [np.random.rand() + 1j * np.random.rand() for _ in range(3)]
beta = [np.random.rand() + 1j * np.random.rand() for _ in range(3)]

qc_mnist = quantum_convolution(q_image, q_kernels, alpha, beta)
qc_stride_mnist = quantum_convolution_stride(q_image, q_kernels[0], stride=2)
qc_pad_mnist = quantum_convolution_pad(q_image, q_kernels[0], pad_width=2)
qc_bn_pool_mnist = quantum_convolution_bn(q_image, q_kernels[0], stride=2, pool_size=14)

qc_lenna = quantum_convolution(q_lenna_image, q_kernels, alpha, beta)
qc_stride_lenna = quantum_convolution_stride(q_lenna_image, q_kernels[0], stride=2)
qc_pad_lenna = quantum_convolution_pad(q_lenna_image, q_kernels[0], pad_width=2)
qc_bn_pool_lenna = quantum_convolution_bn(q_lenna_image, q_kernels[0], stride=2, pool_size=14)

qc_download = quantum_convolution(q_download_image, q_kernels, alpha, beta)
qc_stride_download = quantum_convolution_stride(q_download_image, q_kernels[0], stride=2)
qc_pad_download = quantum_convolution_pad(q_download_image, q_kernels[0], pad_width=2)
qc_bn_pool_download = quantum_convolution_bn(q_download_image, q_kernels[0], stride=2, pool_size=14)

fig, axs = plt.subplots(3, 4, figsize=(20, 15))

axs[0, 0].imshow(q_image, cmap='viridis')
axs[0, 0].set_title('Original MNIST Quantum Image')
axs[0, 1].imshow(np.abs(qc_mnist), cmap='viridis')
axs[0, 1].set_title('Quantum Convolution with Multiple Kernels (MNIST)')
axs[0, 2].imshow(np.abs(qc_stride_mnist), cmap='viridis')
axs[0, 2].set_title('Quantum Convolution with Stride (MNIST)')
axs[0, 3].imshow(np.abs(qc_pad_mnist), cmap='viridis')
axs[0, 3].set_title('Quantum Convolution with Padding (MNIST)')

axs[1, 0].imshow(q_lenna_image, cmap='viridis')
axs[1, 0].set_title('Original Lenna Image')
axs[1, 1].imshow(np.abs(qc_lenna), cmap='viridis')
axs[1, 1].set_title('Quantum Convolution with Multiple Kernels (Lenna)')
axs[1, 2].imshow(np.abs(qc_stride_lenna), cmap='viridis')
axs[1, 2].set_title('Quantum Convolution with Stride (Lenna)')
axs[1, 3].imshow(np.abs(qc_pad_lenna), cmap='viridis')
axs[1, 3].set_title('Quantum Convolution with Padding (Lenna)')

axs[2, 0].imshow(q_download_image, cmap='viridis')
axs[2, 0].set_title('Original Download Image')
axs[2, 1].imshow(np.abs(qc_download), cmap='viridis')
axs[2, 1].set_title('Quantum Convolution with Multiple Kernels (Download)')
axs[2, 2].imshow(np.abs(qc_stride_download), cmap='viridis')
axs[2, 2].set_title('Quantum Convolution with Stride (Download)')
axs[2, 3].imshow(np.abs(qc_pad_download), cmap='viridis')
axs[2, 3].set_title('Quantum Convolution with Padding (Download)')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from PIL import Image
import cv2

# Function for quantum convolution with multiple kernels
def quantum_convolution(q_i, q_kernels, alpha, beta):
    n = len(q_kernels)
    Qc_sum = np.zeros_like(q_i, dtype=np.complex128)
    for m in range(n):
        kernel = q_kernels[m]
        alpha_m = alpha[m]
        beta_m = beta[m]
        kernel_size = kernel.shape
        padded_image = np.pad(q_i, [(kernel_size[0]//2, kernel_size[0]//2), (kernel_size[1]//2, kernel_size[1]//2)], mode='constant')
        conv_result = np.zeros_like(q_i, dtype=np.complex128)
        for i in range(q_i.shape[0]):
            for j in range(q_i.shape[1]):
                conv_result[i, j] = np.sum(
                    padded_image[i:i+kernel_size[0], j:j+kernel_size[1]] * kernel
                )
        Qc_sum += alpha_m * beta_m * conv_result
    return Qc_sum

# Function for quantum convolution with stride
def quantum_convolution_stride(q_i, q_k, stride):
    q_i_stride = q_i[::stride, ::stride]
    kernel_size = q_k.shape[0]
    padded_image = np.pad(q_i_stride, [(kernel_size//2, kernel_size//2), (kernel_size//2, kernel_size//2)], mode='constant')
    Qc_stride = np.zeros_like(q_i_stride, dtype=np.complex128)
    for i in range(q_i_stride.shape[0]):
        for j in range(q_i_stride.shape[1]):
            Qc_stride[i, j] = np.sum(
                padded_image[i:i+kernel_size, j:j+kernel_size] * q_k
            )
    return Qc_stride

# Function for padding image
def pad_image(q_i, pad_width):
    return np.pad(q_i, pad_width, mode='constant')

# Function for quantum convolution with padding
def quantum_convolution_pad(q_i, q_k, pad_width):
    q_i_padded = pad_image(q_i, ((pad_width, pad_width), (pad_width, pad_width)))
    kernel_size = q_k.shape[0]
    Qc_pad = np.zeros_like(q_i_padded, dtype=np.complex128)
    for i in range(q_i.shape[0]):
        for j in range(q_i.shape[1]):
            Qc_pad[i, j] = np.sum(
                q_i_padded[i:i+kernel_size, j:j+kernel_size] * q_k
            )
    return Qc_pad

# Function for batch normalization
def batch_norm(q_i, epsilon=1e-5):
    mu = np.mean(q_i)
    sigma2 = np.var(q_i)
    q_bn = (q_i - mu) / np.sqrt(sigma2 + epsilon)
    return q_bn

# Function for quantum convolution with batch normalization and pooling
def quantum_convolution_bn(q_i, q_k, stride, pool_size):
    q_i_bn = batch_norm(q_i)
    q_i_pooled = q_i_bn[:pool_size, :pool_size]
    Qc_bn_pool = quantum_convolution_stride(q_i_pooled, q_k, stride)
    return Qc_bn_pool

# Data transformations for MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
image, _ = trainset[0]
q_image = np.abs(image.numpy()[0])

# Load and preprocess Lenna image
lenna_image = cv2.imread('C:/Users/ACER/Desktop/lena_256.jpg', cv2.IMREAD_GRAYSCALE)
lenna_image = cv2.resize(lenna_image, (28, 28))
q_lenna_image = lenna_image / 255.0

# Load and preprocess download image
download_image = cv2.imread('C:/Users/ACER/Desktop/download.jpg', cv2.IMREAD_GRAYSCALE)
download_image = cv2.resize(download_image, (28, 28))
q_download_image = download_image / 255.0

# Define quantum kernels and coefficients
q_kernels = [np.random.rand(3, 3) for _ in range(3)]
alpha = [np.random.rand() + 1j * np.random.rand() for _ in range(3)]
beta = [np.random.rand() + 1j * np.random.rand() for _ in range(3)]

# Apply quantum convolution methods
qc_mnist = quantum_convolution(q_image, q_kernels, alpha, beta)
qc_stride_mnist = quantum_convolution_stride(q_image, q_kernels[0], stride=2)
qc_pad_mnist = quantum_convolution_pad(q_image, q_kernels[0], pad_width=2)
qc_bn_pool_mnist = quantum_convolution_bn(q_image, q_kernels[0], stride=2, pool_size=14)

qc_lenna = quantum_convolution(q_lenna_image, q_kernels, alpha, beta)
qc_stride_lenna = quantum_convolution_stride(q_lenna_image, q_kernels[0], stride=2)
qc_pad_lenna = quantum_convolution_pad(q_lenna_image, q_kernels[0], pad_width=2)
qc_bn_pool_lenna = quantum_convolution_bn(q_lenna_image, q_kernels[0], stride=2, pool_size=14)

qc_download = quantum_convolution(q_download_image, q_kernels, alpha, beta)
qc_stride_download = quantum_convolution_stride(q_download_image, q_kernels[0], stride=2)
qc_pad_download = quantum_convolution_pad(q_download_image, q_kernels[0], pad_width=2)
qc_bn_pool_download = quantum_convolution_bn(q_download_image, q_kernels[0], stride=2, pool_size=14)

# Plot results
fig, axs = plt.subplots(3, 5, figsize=(24, 15))

# Plot MNIST results
axs[0, 0].imshow(q_image, cmap='viridis')
axs[0, 0].set_title('Original MNIST Quantum Image')
axs[0, 1].imshow(np.abs(qc_mnist), cmap='viridis')
axs[0, 1].set_title('Quantum Convolution with Multiple Kernels (MNIST)')
axs[0, 2].imshow(np.abs(qc_stride_mnist), cmap='viridis')
axs[0, 2].set_title('Quantum Convolution with Stride (MNIST)')
axs[0, 3].imshow(np.abs(qc_pad_mnist), cmap='viridis')
axs[0, 3].set_title('Quantum Convolution with Padding (MNIST)')
axs[0, 4].imshow(np.abs(qc_bn_pool_mnist), cmap='viridis')
axs[0, 4].set_title('Quantum Convolution with BN (MNIST)')

# Plot Lenna results
axs[1, 0].imshow(q_lenna_image, cmap='viridis')
axs[1, 0].set_title('Original Lenna Image')
axs[1, 1].imshow(np.abs(qc_lenna), cmap='viridis')
axs[1, 1].set_title('Quantum Convolution with Multiple Kernels (Lenna)')
axs[1, 2].imshow(np.abs(qc_stride_lenna), cmap='viridis')
axs[1, 2].set_title('Quantum Convolution with Stride (Lenna)')
axs[1, 3].imshow(np.abs(qc_pad_lenna), cmap='viridis')
axs[1, 3].set_title('Quantum Convolution with Padding (Lenna)')
axs[1, 4].imshow(np.abs(qc_bn_pool_lenna), cmap='viridis')
axs[1, 4].set_title('Quantum Convolution with BN (Lenna)')

# Plot Download image results
axs[2, 0].imshow(q_download_image, cmap='viridis')
axs[2, 0].set_title('Original Pepper Image')
axs[2, 1].imshow(np.abs(qc_download), cmap='viridis')
axs[2, 1].set_title('Quantum Convolution with Multiple Kernels')
axs[2, 2].imshow(np.abs(qc_stride_download), cmap='viridis')
axs[2, 2].set_title('Quantum Convolution with Stride')
axs[2, 3].imshow(np.abs(qc_pad_download), cmap='viridis')
axs[2, 3].set_title('Quantum Convolution with Padding')
axs[2, 4].imshow(np.abs(qc_bn_pool_download), cmap='viridis')
axs[2, 4].set_title('Quantum Convolution with BN')

plt.tight_layout()
plt.savefig("quantum_image_processing.svg")
plt.show()


In [ ]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Define the quantum convolution function with weights
def quantum_convolution_weighted(q_i, q_k_list, weights):
    result = torch.zeros_like(q_i)
    for q_k, weight in zip(q_k_list, weights):
        padding = (q_k.shape[-1] // 2, q_k.shape[-2] // 2)
        result += weight * F.conv2d(q_i, q_k, padding=padding)
    return result

# Plot the original and processed images
def plot_images(original, processed, title):
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(original.squeeze(), cmap='gray')
    axs[0].set_title('Original Image')
    axs[1].imshow(processed.squeeze().detach().numpy(), cmap='gray')
    axs[1].set_title(title)
    plt.show()

# Main function to apply quantum convolution
def main():
    # Load the MNIST dataset
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
    mnist_image, _ = trainset[0]
    q_mnist_image = mnist_image.unsqueeze(0)  # Add batch dimension

    # Load and process the Lenna image
    lenna_image = cv2.imread('C:/Users/ACER/Desktop/lena_256.jpg', cv2.IMREAD_GRAYSCALE)
    lenna_image = cv2.resize(lenna_image, (28, 28))
    q_lenna_image = torch.tensor(lenna_image / 255.0, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Add channel and batch dimensions

    # Load and process the download image
    download_image = cv2.imread('C:/Users/ACER/Desktop/download.jpg', cv2.IMREAD_GRAYSCALE)
    download_image = cv2.resize(download_image, (28, 28))
    q_download_image = torch.tensor(download_image / 255.0, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Add channel and batch dimensions

    # Define quantum kernels
    kernel_1 = torch.randn(1, 1, 3, 3)  # Example kernel
    kernel_2 = torch.randn(1, 1, 3, 3)  # Example kernel
    kernel_3 = torch.randn(1, 1, 3, 3)  # Example kernel

    # Define weights
    weights = [0.2, 0.3, 0.5]

    # Apply quantum convolution with weighted kernels to MNIST image
    processed_mnist_weighted = quantum_convolution_weighted(q_mnist_image, [kernel_1, kernel_2, kernel_3], weights)
    plot_images(q_mnist_image.squeeze(), processed_mnist_weighted, 'Weighted Quantum Convolution (MNIST)')

    # Apply quantum convolution with weighted kernels to Lenna image
    processed_lenna_weighted = quantum_convolution_weighted(q_lenna_image, [kernel_1, kernel_2, kernel_3], weights)
    plot_images(q_lenna_image.squeeze(), processed_lenna_weighted, 'Weighted Quantum Convolution (Lenna)')

    # Apply quantum convolution with weighted kernels to download image
    processed_download_weighted = quantum_convolution_weighted(q_download_image, [kernel_1, kernel_2, kernel_3], weights)
    plot_images(q_download_image.squeeze(), processed_download_weighted, 'Weighted Quantum Convolution (Download)')

if __name__ == "__main__":
    main()


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
import cv2

# Define the transform for the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
image, _ = trainset[0]
q_image = np.abs(image.numpy()[0])

# Load and process the Lenna image
lenna_image = cv2.imread('C:/Users/ACER/Desktop/lena_256.jpg', cv2.IMREAD_GRAYSCALE)
lenna_image = cv2.resize(lenna_image, (28, 28))
q_lenna_image = lenna_image / 255.0

# Load and process another image
download_image = cv2.imread('C:/Users/ACER/Desktop/download.jpg', cv2.IMREAD_GRAYSCALE)
download_image = cv2.resize(download_image, (28, 28))
q_download_image = download_image / 255.0

# Define a function to apply the weighted quantum convolution
def quantum_convolution_weighted(q_i, q_k, weights):
    result = np.zeros_like(q_i)
    for k, w in zip(q_k, weights):
        result += w * (k * q_i)
    return result

# Define a function to apply the separable quantum convolution
def quantum_convolution_separable(q_i, q_kx, q_ky):
    result = np.zeros_like(q_i)
    for kx, ky in zip(q_kx, q_ky):
        result += np.outer(kx, ky) * q_i
    return result

# Define a function to apply the dilated quantum convolution
def quantum_convolution_dilated(q_i, q_k, d):
    result = np.zeros_like(q_i)
    for k in q_k:
        dilated_k = cv2.dilate(k, np.ones((d, d), np.uint8))
        result += dilated_k * q_i
    return result

# Define a function to apply the grouped quantum convolution
def quantum_convolution_grouped(q_i, q_kgroups):
    result = np.zeros_like(q_i)
    for group in q_kgroups:
        group_result = np.zeros_like(q_i)
        for k in group:
            group_result += k * q_i
        result += group_result
    return result

# Example kernels and weights
kernels = [np.random.rand(28, 28) for _ in range(3)]
weights = [0.3, 0.5, 0.2]
kx = [np.random.rand(28) for _ in range(3)]
ky = [np.random.rand(28) for _ in range(3)]
groups = [[np.random.rand(28, 28) for _ in range(2)], [np.random.rand(28, 28) for _ in range(2)]]
dilation_rate = 2

# Apply the quantum convolution operations to the MNIST image
q_weighted = quantum_convolution_weighted(q_image, kernels, weights)
q_separable = quantum_convolution_separable(q_image, kx, ky)
q_dilated = quantum_convolution_dilated(q_image, kernels, dilation_rate)
q_grouped = quantum_convolution_grouped(q_image, groups)

# Plot the results for MNIST image
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
axes[0].imshow(q_image, cmap='gray')
axes[0].set_title('Original MNIST Image')
axes[1].imshow(q_weighted, cmap='gray')
axes[1].set_title('Weighted Quantum Convolution')
axes[2].imshow(q_separable, cmap='gray')
axes[2].set_title('Separable Quantum Convolution')
axes[3].imshow(q_dilated, cmap='gray')
axes[3].set_title('Dilated Quantum Convolution')
axes[4].imshow(q_grouped, cmap='gray')
axes[4].set_title('Grouped Quantum Convolution')
plt.show()

# Apply the quantum convolution operations to the Lenna image
q_weighted_lenna = quantum_convolution_weighted(q_lenna_image, kernels, weights)
q_separable_lenna = quantum_convolution_separable(q_lenna_image, kx, ky)
q_dilated_lenna = quantum_convolution_dilated(q_lenna_image, kernels, dilation_rate)
q_grouped_lenna = quantum_convolution_grouped(q_lenna_image, groups)

# Plot the results for Lenna image
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
axes[0].imshow(q_lenna_image, cmap='gray')
axes[0].set_title('Original Lenna Image')
axes[1].imshow(q_weighted_lenna, cmap='gray')
axes[1].set_title('Weighted Quantum Convolution')
axes[2].imshow(q_separable_lenna, cmap='gray')
axes[2].set_title('Separable Quantum Convolution')
axes[3].imshow(q_dilated_lenna, cmap='gray')
axes[3].set_title('Dilated Quantum Convolution')
axes[4].imshow(q_grouped_lenna, cmap='gray')
axes[4].set_title('Grouped Quantum Convolution')
plt.show()

# Apply the quantum convolution operations to the downloaded image
q_weighted_download = quantum_convolution_weighted(q_download_image, kernels, weights)
q_separable_download = quantum_convolution_separable(q_download_image, kx, ky)
q_dilated_download = quantum_convolution_dilated(q_download_image, kernels, dilation_rate)
q_grouped_download = quantum_convolution_grouped(q_download_image, groups)

# Plot the results for downloaded image
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
axes[0].imshow(q_download_image, cmap='gray')
axes[0].set_title('Original Downloaded Image')
axes[1].imshow(q_weighted_download, cmap='gray')
axes[1].set_title('Weighted Quantum Convolution')
axes[2].imshow(q_separable_download, cmap='gray')
axes[2].set_title('Separable Quantum Convolution')
axes[3].imshow(q_dilated_download, cmap='gray')
axes[3].set_title('Dilated Quantum Convolution')
axes[4].imshow(q_grouped_download, cmap='gray')
axes[4].set_title('Grouped Quantum Convolution')
plt.show()


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from PIL import Image
import matplotlib.pyplot as plt

# Helper functions for image loading and preprocessing
def load_image(image_path, size=(28, 28)):
    image = Image.open(image_path).convert('L')
    image = image.resize(size)
    image = np.array(image)
    return image

def plot_images(images, titles, cmap='viridis'):
    num_images = len(images)
    cols = 3
    rows = (num_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten()
    
    for i in range(num_images):
        axes[i].imshow(images[i], cmap=cmap)
        axes[i].set_title(titles[i])
        axes[i].axis('off')
    
    # Turn off axes for any remaining empty subplots
    for i in range(num_images, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Padding methods
def cyclic_padding(image, pad_width):
    return np.pad(image, pad_width=pad_width, mode='wrap')

def symmetric_padding(image, pad_width):
    return np.pad(image, pad_width=pad_width, mode='symmetric')

# Quantum Convolution (QuC) with padding
def quantum_convolution(image, kernel, padding, method='cyclic'):
    if method == 'cyclic':
        padded_image = cyclic_padding(image, padding)
    elif method == 'symmetric':
        padded_image = symmetric_padding(image, padding)
    
    # Convolution operation
    kernel_size = kernel.shape[0]
    output_size = (padded_image.shape[0] - kernel_size + 1, padded_image.shape[1] - kernel_size + 1)
    output = np.zeros(output_size)
    
    for i in range(output_size[0]):
        for j in range(output_size[1]):
            output[i, j] = np.sum(padded_image[i:i+kernel_size, j:j+kernel_size] * kernel)
    
    return output

# Stride and pooling operations
def conv_with_stride(image, kernel, stride):
    kernel_size = kernel.shape[0]
    output_size = ((image.shape[0] - kernel_size) // stride + 1, (image.shape[1] - kernel_size) // stride + 1)
    output = np.zeros(output_size)
    
    for i in range(0, output_size[0] * stride, stride):
        for j in range(0, output_size[1] * stride, stride):
            if i + kernel_size <= image.shape[0] and j + kernel_size <= image.shape[1]:
                output[i // stride, j // stride] = np.sum(image[i:i+kernel_size, j:j+kernel_size] * kernel)
    
    return output

def conv_with_padding_and_stride(image, kernel, padding, stride):
    padded_image = cyclic_padding(image, padding)
    return conv_with_stride(padded_image, kernel, stride)

# Batch Normalization (BN) operation
def batch_normalization(image, mean, variance, epsilon=1e-5):
    return (image - mean) / np.sqrt(variance + epsilon)

# Weighted kernels and grouped kernels
def apply_weighted_kernels(image, kernels, weights):
    result = np.zeros(image.shape)
    for kernel, weight in zip(kernels, weights):
        result += weight * quantum_convolution(image, kernel, padding=0)
    return result

def apply_grouped_kernels(image, grouped_kernels):
    results = []
    for group in grouped_kernels:
        result = np.zeros(image.shape)
        for kernel in group:
            result += quantum_convolution(image, kernel, padding=0)
        results.append(result)
    return results

# Amplitude Amplification (example placeholder)
def amplitude_amplification(image):
    return image  # Placeholder for actual amplitude amplification implementation

# Entanglement (example placeholder)
def entanglement_operation(image, kernel):
    return quantum_convolution(image, kernel, padding=0)  # Placeholder for actual entanglement implementation

# QFT and its inverse
def quantum_fourier_transform(image):
    # Placeholder for actual QFT implementation
    return np.fft.fft2(image)

def inverse_quantum_fourier_transform(image):
    # Placeholder for actual inverse QFT implementation
    return np.fft.ifft2(image)

# Full QuC with padding and QFT integration
def full_quantum_convolution(image, kernel, padding):
    padded_image = cyclic_padding(image, padding)
    padded_kernel = cyclic_padding(kernel, padding)
    qft_image = quantum_fourier_transform(padded_image)
    qft_kernel = quantum_fourier_transform(padded_kernel)
    qft_result = qft_image * qft_kernel
    result = inverse_quantum_fourier_transform(qft_result)
    return np.real(result)

# Load MNIST dataset and preprocess
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
mnist_image, _ = trainset[0]
q_mnist_image = np.abs(mnist_image.numpy()[0])

# Load and process the Lenna image
lenna_image_path = 'C:/Users/ACER/Desktop/lena_256.jpg'
lenna_image = load_image(lenna_image_path)
q_lenna_image = lenna_image / 255.0

# Load and process the Pepper image
pepper_image_path = 'C:/Users/ACER/Desktop/pepper.jpg'
pepper_image = load_image(pepper_image_path)
q_pepper_image = pepper_image / 255.0

# Example of using the defined functions
sample_kernel = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])  # Example kernel
padding = 5
stride = 2

# Processing with padding and stride
padded_mnist_image = cyclic_padding(q_mnist_image, pad_width=padding)
conv_result_mnist = quantum_convolution(q_mnist_image, sample_kernel, padding=padding)
conv_stride_result_mnist = conv_with_padding_and_stride(q_mnist_image, sample_kernel, padding=padding, stride=stride)

padded_lenna_image = cyclic_padding(q_lenna_image, pad_width=padding)
conv_result_lenna = quantum_convolution(q_lenna_image, sample_kernel, padding=padding)
conv_stride_result_lenna = conv_with_padding_and_stride(q_lenna_image, sample_kernel, padding=padding, stride=stride)

padded_pepper_image = cyclic_padding(q_pepper_image, pad_width=padding)
conv_result_pepper = quantum_convolution(q_pepper_image, sample_kernel, padding=padding)
conv_stride_result_pepper = conv_with_padding_and_stride(q_pepper_image, sample_kernel, padding=padding, stride=stride)

# Display results in a grid
images = [
    q_mnist_image, conv_result_mnist, conv_stride_result_mnist,
    q_lenna_image, conv_result_lenna, conv_stride_result_lenna,
    q_pepper_image, conv_result_pepper, conv_stride_result_pepper
]
titles = [
    'MNIST Image', 'MNIST Convolution Result', 'MNIST Convolution with Padding and Stride Result',
    'Lenna Image', 'Lenna Convolution Result', 'Lenna Convolution with Padding and Stride Result',
    'Pepper Image', 'Pepper Convolution Result', 'Pepper Convolution with Padding and Stride Result'
]

plot_images(images, titles, cmap='viridis')


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from PIL import Image
import matplotlib.pyplot as plt

# Helper functions for image loading and preprocessing
def load_image(image_path, size=(28, 28)):
    image = Image.open(image_path).convert('L')
    image = image.resize(size)
    image = np.array(image)
    return image

def plot_images(images, titles, cmap='viridis'):
    num_images = len(images)
    cols = 3
    rows = (num_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten()
    
    for i in range(num_images):
        axes[i].imshow(images[i], cmap=cmap)
        axes[i].set_title(titles[i])
        axes[i].axis('off')
    
    # Turn off axes for any remaining empty subplots
    for i in range(num_images, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Padding methods
def cyclic_padding(image, pad_width):
    return np.pad(image, pad_width=pad_width, mode='wrap')

# Quantum Fourier Transform (QFT) and its inverse
def quantum_fourier_transform(image):
    return np.fft.fft2(image)

def inverse_quantum_fourier_transform(image):
    return np.fft.ifft2(image)

# Quantum Convolution with Pre-padding and QFT
def quantum_convolution_with_pre_padding(image, kernel, padding):
    padded_image = cyclic_padding(image, pad_width=padding)
    kernel_size = kernel.shape
    padded_kernel = cyclic_padding(kernel, pad_width=((padded_image.shape[0] - kernel_size[0]) // 2, (padded_image.shape[1] - kernel_size[1]) // 2))
    
    # Ensure the padded kernel and image have the same size
    padded_kernel = np.pad(kernel, pad_width=((0, padded_image.shape[0] - kernel_size[0]), (0, padded_image.shape[1] - kernel_size[1])), mode='constant', constant_values=0)
    
    qft_image = quantum_fourier_transform(padded_image)
    qft_kernel = quantum_fourier_transform(padded_kernel)
    
    qft_result = qft_image * qft_kernel
    result = inverse_quantum_fourier_transform(qft_result)
    
    return np.real(result)

# Load MNIST dataset and preprocess
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
mnist_image, _ = trainset[0]
q_mnist_image = np.abs(mnist_image.numpy()[0])

# Load and process the Lenna image
lenna_image_path = 'C:/Users/ACER/Desktop/lena_256.jpg'
lenna_image = load_image(lenna_image_path)
q_lenna_image = lenna_image / 255.0

# Load and process the Pepper image
pepper_image_path = 'C:/Users/ACER/Desktop/pepper.jpg'
pepper_image = load_image(pepper_image_path)
q_pepper_image = pepper_image / 255.0

# Example of using the defined functions
sample_kernel = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])  # Example kernel
padding = 10

# Quantum convolution with pre-padding
conv_result_mnist = quantum_convolution_with_pre_padding(q_mnist_image, sample_kernel, padding=padding)
conv_result_lenna = quantum_convolution_with_pre_padding(q_lenna_image, sample_kernel, padding=padding)
conv_result_pepper = quantum_convolution_with_pre_padding(q_pepper_image, sample_kernel, padding=padding)

# Display results in a grid
images = [
    q_mnist_image, conv_result_mnist,
    q_lenna_image, conv_result_lenna,
    q_pepper_image, conv_result_pepper
]
titles = [
    'MNIST Image', 'MNIST Convolution Result',
    'Lenna Image', 'Lenna Convolution Result',
    'Pepper Image', 'Pepper Convolution Result'
]

plot_images(images, titles, cmap='viridis')


In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

# Define the transform for the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
mnist_image, _ = trainset[0]
mnist_image = mnist_image.numpy()[0]

# Load and process the Lena image
lena_image = Image.open('C:/Users/ACER/Desktop/lena_256.jpg').convert('L')
lena_image = lena_image.resize((28, 28))
lena_image = np.array(lena_image) / 255.0

# Load and process the Pepper image
pepper_image = Image.open('C:/Users/ACER/Desktop/pepper.jpg').convert('L')
pepper_image = pepper_image.resize((28, 28))
pepper_image = np.array(pepper_image) / 255.0

# Example kernel (You should define your own kernel)
sample_kernel = np.ones((3, 3)) / 9  # Simple average kernel for demonstration
padding = 25  # Example padding, adjust as needed

def quantum_fourier_transform(image):
    return np.fft.fft2(image)

def inverse_quantum_fourier_transform(image):
    return np.fft.ifft2(image)

def quantum_convolution_with_pre_padding(image, kernel, padding):
    # Calculate new padded sizes
    padded_size = (image.shape[0] + 2 * padding, image.shape[1] + 2 * padding)
    
    # Apply padding to image and kernel
    padded_image = np.pad(image, ((padding, padding), (padding, padding)), mode='constant')
    padded_kernel = np.pad(kernel, ((padding, padding), (padding, padding)), mode='constant')
    
    # Resize kernel to match the size of the padded image
    padded_kernel = np.resize(padded_kernel, padded_image.shape)
    
    # Apply QFT
    qft_image = quantum_fourier_transform(padded_image)
    qft_kernel = quantum_fourier_transform(padded_kernel)
    
    # Convolution in frequency domain
    qft_result = qft_image * qft_kernel
    
    # Inverse QFT
    result = inverse_quantum_fourier_transform(qft_result)
    
    return np.abs(result)

# Perform quantum convolution with pre-padding
conv_result_mnist = quantum_convolution_with_pre_padding(mnist_image, sample_kernel, padding)
conv_result_lena = quantum_convolution_with_pre_padding(lena_image, sample_kernel, padding)
conv_result_pepper = quantum_convolution_with_pre_padding(pepper_image, sample_kernel, padding)

# Plotting
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# Original Images
axs[0, 0].imshow(mnist_image, cmap='viridis')
axs[0, 0].set_title('MNIST Image')
axs[0, 0].axis('off')

axs[0, 1].imshow(lena_image, cmap='viridis')
axs[0, 1].set_title('Lena Image')
axs[0, 1].axis('off')

# Convolution Results
axs[1, 0].imshow(conv_result_mnist, cmap='viridis')
axs[1, 0].set_title('MNIST Convolution Result')
axs[1, 0].axis('off')

axs[1, 1].imshow(conv_result_lena, cmap='viridis')
axs[1, 1].set_title('Lena Convolution Result')
axs[1, 1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import datasets, transforms

# Define the transform for the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
image, _ = trainset[0]
q_image = np.abs(image.numpy()[0])

# Load and process the Lena image
def load_image(path, size=(28, 28)):
    image = Image.open(path).convert('L')
    image = image.resize(size)
    return np.array(image) / 255.0

lenna_image = load_image('C:/Users/ACER/Desktop/lena_256.jpg')
download_image = load_image('C:/Users/ACER/Desktop/pepper.jpg')

# Define a function to apply the weighted quantum convolution
def quantum_convolution_weighted(q_i, q_k, weights):
    result = np.zeros_like(q_i)
    for k, w in zip(q_k, weights):
        result += w * (k * q_i)
    return result

# Define a function to apply the separable quantum convolution
def quantum_convolution_separable(q_i, q_kx, q_ky):
    result = np.zeros_like(q_i)
    for kx, ky in zip(q_kx, q_ky):
        result += np.outer(kx, ky) * q_i
    return result

# Define a function to apply the dilated quantum convolution
def quantum_convolution_dilated(q_i, q_k, d):
    result = np.zeros_like(q_i)
    kernel_size = q_k[0].shape  # Assuming all kernels have the same size

    for k in q_k:
        dilated_k = np.zeros_like(q_i)
        for i in range(0, q_i.shape[0] - kernel_size[0] + 1, d):
            for j in range(0, q_i.shape[1] - kernel_size[1] + 1, d):
                dilated_k[i:i+kernel_size[0], j:j+kernel_size[1]] = k
        result += dilated_k * q_i
    return result

# Define a function to apply the grouped quantum convolution
def quantum_convolution_grouped(q_i, q_kgroups):
    result = np.zeros_like(q_i)
    for group in q_kgroups:
        group_result = np.zeros_like(q_i)
        for k in group:
            group_result += k * q_i
        result += group_result
    return result

# Example kernels and weights
kernels = [np.random.rand(28, 28) for _ in range(3)]
weights = [0.3, 0.5, 0.2]
kx = [np.random.rand(28) for _ in range(3)]
ky = [np.random.rand(28) for _ in range(3)]
groups = [[np.random.rand(28, 28) for _ in range(2)], [np.random.rand(28, 28) for _ in range(2)]]
dilation_rate = 2

# Apply the quantum convolution operations to the MNIST image
q_weighted = quantum_convolution_weighted(q_image, kernels, weights)
q_separable = quantum_convolution_separable(q_image, kx, ky)
q_dilated = quantum_convolution_dilated(q_image, kernels, dilation_rate)
q_grouped = quantum_convolution_grouped(q_image, groups)

# Plot the results for MNIST image
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
axes[0].imshow(q_image, cmap='gray')
axes[0].set_title('Original MNIST Image')
axes[1].imshow(q_weighted, cmap='gray')
axes[1].set_title('Weighted Quantum Convolution')
axes[2].imshow(q_separable, cmap='gray')
axes[2].set_title('Separable Quantum Convolution')
axes[3].imshow(q_dilated, cmap='gray')
axes[3].set_title('Dilated Quantum Convolution')
axes[4].imshow(q_grouped, cmap='gray')
axes[4].set_title('Grouped Quantum Convolution')
plt.show()

# Apply the quantum convolution operations to the Lena image
q_weighted_lenna = quantum_convolution_weighted(lenna_image, kernels, weights)
q_separable_lenna = quantum_convolution_separable(lenna_image, kx, ky)
q_dilated_lenna = quantum_convolution_dilated(lenna_image, kernels, dilation_rate)
q_grouped_lenna = quantum_convolution_grouped(lenna_image, groups)

# Plot the results for Lenna image
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
axes[0].imshow(lenna_image, cmap='gray')
axes[0].set_title('Original Lenna Image')
axes[1].imshow(q_weighted_lenna, cmap='gray')
axes[1].set_title('Weighted Quantum Convolution')
axes[2].imshow(q_separable_lenna, cmap='gray')
axes[2].set_title('Separable Quantum Convolution')
axes[3].imshow(q_dilated_lenna, cmap='gray')
axes[3].set_title('Dilated Quantum Convolution')
axes[4].imshow(q_grouped_lenna, cmap='gray')
axes[4].set_title('Grouped Quantum Convolution')
plt.show()

# Apply the quantum convolution operations to the downloaded image
q_weighted_download = quantum_convolution_weighted(download_image, kernels, weights)
q_separable_download = quantum_convolution_separable(download_image, kx, ky)
q_dilated_download = quantum_convolution_dilated(download_image, kernels, dilation_rate)
q_grouped_download = quantum_convolution_grouped(download_image, groups)

# Plot the results for downloaded image
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
axes[0].imshow(download_image, cmap='gray')
axes[0].set_title('Original Downloaded Image')
axes[1].imshow(q_weighted_download, cmap='gray')
axes[1].set_title('Weighted Quantum Convolution')
axes[2].imshow(q_separable_download, cmap='gray')
axes[2].set_title('Separable Quantum Convolution')
axes[3].imshow(q_dilated_download, cmap='gray')
axes[3].set_title('Dilated Quantum Convolution')
axes[4].imshow(q_grouped_download, cmap='gray')
axes[4].set_title('Grouped Quantum Convolution')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import datasets, transforms

# Define the transform for the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
image, _ = trainset[0]
q_image = np.abs(image.numpy()[0])

# Load and process the Lena image
def load_image(path, size=(28, 28)):
    image = Image.open(path).convert('L')
    image = image.resize(size)
    return np.array(image) / 255.0

lenna_image = load_image('C:/Users/ACER/Desktop/lena_256.jpg')
download_image = load_image('C:/Users/ACER/Desktop/pepper.jpg')

# Define a function to apply the weighted quantum convolution
def quantum_convolution_weighted(q_i, q_k, weights):
    result = np.zeros_like(q_i)
    for k, w in zip(q_k, weights):
        result += w * (k * q_i)
    return result

# Define a function to apply the separable quantum convolution
def quantum_convolution_separable(q_i, q_kx, q_ky):
    result = np.zeros_like(q_i)
    for kx, ky in zip(q_kx, q_ky):
        result += np.outer(kx, ky) * q_i
    return result

# Define a function to apply the dilated quantum convolution
def quantum_convolution_dilated(q_i, q_k, d):
    result = np.zeros_like(q_i)
    kernel_size = q_k[0].shape  # Assuming all kernels have the same size

    for k in q_k:
        dilated_k = np.zeros_like(q_i)
        for i in range(0, q_i.shape[0] - kernel_size[0] + 1, d):
            for j in range(0, q_i.shape[1] - kernel_size[1] + 1, d):
                dilated_k[i:i+kernel_size[0], j:j+kernel_size[1]] = k
        result += dilated_k * q_i
    return result

# Define a function to apply the grouped quantum convolution
def quantum_convolution_grouped(q_i, q_kgroups):
    result = np.zeros_like(q_i)
    for group in q_kgroups:
        group_result = np.zeros_like(q_i)
        for k in group:
            group_result += k * q_i
        result += group_result
    return result

# Example kernels and weights
kernels = [np.random.rand(28, 28) for _ in range(3)]
weights = [0.3, 0.5, 0.2]
kx = [np.random.rand(28) for _ in range(3)]
ky = [np.random.rand(28) for _ in range(3)]
groups = [[np.random.rand(28, 28) for _ in range(2)], [np.random.rand(28, 28) for _ in range(2)]]
dilation_rate = 2

# Apply the quantum convolution operations to the MNIST image
q_weighted = quantum_convolution_weighted(q_image, kernels, weights)
q_separable = quantum_convolution_separable(q_image, kx, ky)
q_dilated = quantum_convolution_dilated(q_image, kernels, dilation_rate)
q_grouped = quantum_convolution_grouped(q_image, groups)

# Plot the results for MNIST image
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
axes[0].imshow(q_image, cmap='viridis')
axes[0].set_title('Original MNIST Image')
axes[1].imshow(q_weighted, cmap='viridis')
axes[1].set_title('Weighted Quantum Convolution')
axes[2].imshow(q_separable, cmap='viridis')
axes[2].set_title('Separable Quantum Convolution')
axes[3].imshow(q_dilated, cmap='viridis')
axes[3].set_title('Dilated Quantum Convolution')
axes[4].imshow(q_grouped, cmap='viridis')
axes[4].set_title('Grouped Quantum Convolution')
plt.show()

# Apply the quantum convolution operations to the Lena image
q_weighted_lenna = quantum_convolution_weighted(lenna_image, kernels, weights)
q_separable_lenna = quantum_convolution_separable(lenna_image, kx, ky)
q_dilated_lenna = quantum_convolution_dilated(lenna_image, kernels, dilation_rate)
q_grouped_lenna = quantum_convolution_grouped(lenna_image, groups)

# Plot the results for Lenna image
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
axes[0].imshow(lenna_image, cmap='viridis')
axes[0].set_title('Original Lenna Image')
axes[1].imshow(q_weighted_lenna, cmap='viridis')
axes[1].set_title('Weighted Quantum Convolution')
axes[2].imshow(q_separable_lenna, cmap='viridis')
axes[2].set_title('Separable Quantum Convolution')
axes[3].imshow(q_dilated_lenna, cmap='viridis')
axes[3].set_title('Dilated Quantum Convolution')
axes[4].imshow(q_grouped_lenna, cmap='viridis')
axes[4].set_title('Grouped Quantum Convolution')
plt.show()

# Apply the quantum convolution operations to the downloaded image
q_weighted_download = quantum_convolution_weighted(download_image, kernels, weights)
q_separable_download = quantum_convolution_separable(download_image, kx, ky)
q_dilated_download = quantum_convolution_dilated(download_image, kernels, dilation_rate)
q_grouped_download = quantum_convolution_grouped(download_image, groups)

# Plot the results for downloaded image
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
axes[0].imshow(download_image, cmap='viridis')
axes[0].set_title('Original Downloaded Image')
axes[1].imshow(q_weighted_download, cmap='viridis')
axes[1].set_title('Weighted Quantum Convolution')
axes[2].imshow(q_separable_download, cmap='viridis')
axes[2].set_title('Separable Quantum Convolution')
axes[3].imshow(q_dilated_download, cmap='viridis')
axes[3].set_title('Dilated Quantum Convolution')
axes[4].imshow(q_grouped_download, cmap='viridis')
axes[4].set_title('Grouped Quantum Convolution')
plt.show()
